In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tqdm import tqdm
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, TFBertModel
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.optimizers import Adam
import numpy as np



In [5]:
import tensorflow as tf

In [4]:
# 데이터 로드
data = pd.read_csv("../crwaling/situation.csv")
data = data.drop(columns=["Unnamed: 0"])

In [ ]:
# BERT 모델 불러오기
bert_model_directory = "bert_model_directory"
bert_model = TFBertModel.from_pretrained(bert_model_directory)
tokenizer = BertTokenizer.from_pretrained(bert_model_directory)


In [ ]:
# 감정 태그를 숫자로 매핑
tag_mapping = {tag: idx for idx, tag in enumerate(data['태그'].unique())}
inverse_tag_mapping = {v: k for k, v in tag_mapping.items()}
data['태그_encoded'] = data['태그'].map(tag_mapping)

# 데이터 분할
X_train,X_val, y_train,y_val = train_test_split(data["가사"],data['태그_encoded'], test_size=0.2, random_state=42)

In [ ]:
tokenized_X_train = X_train.apply(lambda x: tokenizer.encode_plus(x, add_special_tokens=True, return_token_type_ids=True, return_attention_mask=True))
tokenized_X_val = X_val.apply(lambda x: tokenizer.encode_plus(x, add_special_tokens=True, return_token_type_ids=True, return_attention_mask=True))
max_length = 100  # Choose an appropriate maximum length

# Extract Token IDs, Segment IDs, and Attention Mask
token_ids_X_train = [seq['input_ids'] for seq in tokenized_X_train]
segment_ids_X_train = [seq['token_type_ids'] for seq in tokenized_X_train]
attention_mask_X_train = [seq['attention_mask'] for seq in tokenized_X_train]
token_ids_X_val = [seq['input_ids'] for seq in tokenized_X_val]
segment_ids_X_val = [seq['token_type_ids'] for seq in tokenized_X_val]
attention_mask_X_val = [seq['attention_mask'] for seq in tokenized_X_val]

# Pad sequences
padded_token_ids_X_train = tf.keras.preprocessing.sequence.pad_sequences(token_ids_X_train, maxlen=max_length)
padded_segment_ids_X_train = tf.keras.preprocessing.sequence.pad_sequences(segment_ids_X_train, maxlen=max_length)
padded_attention_mask_X_train = tf.keras.preprocessing.sequence.pad_sequences(attention_mask_X_train, maxlen=max_length)
padded_token_ids_X_val = tf.keras.preprocessing.sequence.pad_sequences(token_ids_X_val, maxlen=max_length)
padded_segment_ids_X_val = tf.keras.preprocessing.sequence.pad_sequences(segment_ids_X_val, maxlen=max_length)
padded_attention_mask_X_val = tf.keras.preprocessing.sequence.pad_sequences(attention_mask_X_val, maxlen=max_length)
def create_model(bert_model, max_length, num_classes):
    input_ids = tf.keras.layers.Input(shape=(max_length,), dtype=tf.int32, name="input_ids")
    attention_mask = tf.keras.layers.Input(shape=(max_length,), dtype=tf.int32, name="attention_mask")
    token_type_ids = tf.keras.layers.Input(shape=(max_length,), dtype=tf.int32, name="token_type_ids")

    # Extract BERT output for [CLS] token
    bert_output = bert_model(input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)[0][:, 0, :]

    # Apply dropout
    dropout_output = tf.keras.layers.Dropout(0.2)(bert_output)

    # Output layer for your task
    output_layer = tf.keras.layers.Dense(num_classes, activation='softmax')(dropout_output)

    # Define model inputs and outputs
    inputs = {"input_ids": input_ids, "attention_mask": attention_mask, "token_type_ids": token_type_ids}
    outputs = output_layer

    # Create and compile the model
    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    model.compile(loss='sparse_categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), metrics=['accuracy'])

    return model
model= create_model(bert_model,100,4)

In [ ]:
epochs = 1
batch_size = 16

history = model.fit(
    {"input_ids": padded_token_ids_X_train, "attention_mask": padded_attention_mask_X_train, "token_type_ids": padded_segment_ids_X_train},
    y_train,
    epochs=epochs,
    batch_size=batch_size,
    validation_data=({"input_ids": padded_token_ids_X_val, "attention_mask": padded_attention_mask_X_val, "token_type_ids": padded_segment_ids_X_val}, y_val)
)


